<b> Import Library, find available devices </b>

In [1]:
# Initialization code for 3.3 V SiGe LVR Load Regulation Testing.
# This block identifies and initializes the connected SMUs (Source Measure Units).
# If running for the first time, ensure the SMU names are correctly referenced in the subsequent code block.

# This program sweeps VIN for a fixed VREF and measures total quiescent current (IQ) and VOUT. 
# This program sets up a directory for all measurements taken at a specific temperature by prompting the user for the temperature.
# This program creates a .csv file to save the measurements for a VREF in the temperature directory. 

# The setup assumes the use of two SMUs:
# - One dedicated to VIN sweeps.
# - One Sinks a load current and measures VOUT.

import pyvisa                    # Importing the pyvisa library for instrument communication
import time                      # Importing the time module to handle time-related tasks (e.g., delays)
import matplotlib.pyplot as plt  # Importing matplotlib.pyplot for plotting graphs and visualizing data
import numpy as np               # Importing numpy for numerical operations, particularly with arrays
import pandas as dp              # Importing pandas as dp for data manipulation and analysis
import os                        # Importing os to interact with the operating system, such as handling file paths
import csv                       # Importing csv to read from and write to CSV files
rm = pyvisa.ResourceManager()    # Create a ResourceManager object, which manages communication with instruments
rm.list_resources()              # List all available resources (e.g., connected instruments) managed by pyvisa

('USB0::0x0957::0x5707::MY53802060::INSTR',
 'USB0::0x2A8D::0x0101::MY57508183::INSTR',
 'ASRL3::INSTR',
 'ASRL4::INSTR',
 'GPIB6::6::INSTR',
 'USB0::0x0957::0x0607::MY45002108::0::INSTR',
 'USB0::0x0957::0x1796::MY57231548::0::INSTR',
 'USB0::0x0957::0x1796::MY57231553::0::INSTR',
 'USB0::0x0957::0x17B6::MY56310504::0::INSTR',
 'USB0::0x0957::0x2C07::MY57890777::0::INSTR',
 'USB0::0x2A8D::0x0101::MY54505624::0::INSTR')

In [2]:
#Establishing Connection for VIN SMU
SM_VIN = rm.open_resource('ASRL3::INSTR')  # Open a connection to the instrument with the address 'ASRL3::INSTR' and assign it to SM_VIN
SM_VIN.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
SM_VIN.write_termination = '\n'            # Set the write termination character to a newline. This ensures that each command sent to the instrument is properly terminated.
SM_VIN.baud_rate = 9600                    # Set the baud rate for serial communication to 9600. This is the speed at which data is transmitted over the serial connection.
print(SM_VIN)                              # Print the resource object (SM_VIN) to display its properties
print(SM_VIN.query('*IDN?'))               # Send the '*IDN?' command to the instrument and print its response. This command usually asks the instrument to identify itself (e.g., manufacturer, model number).

#Establishing Connection for VOUT SMU
SM_VOUT = rm.open_resource('ASRL4::INSTR') # Open a connection to the instrument with the address 'ASRL4::INSTR' and assign it to SM_VOUT
SM_VOUT.read_termination = '\n'            # Set the read termination character to a newline for SM_VOUT, similar to SM_VIN
SM_VOUT.write_termination = '\n'           # Set the write termination character to a newline for SM_VOUT, similar to SM_VIN
SM_VOUT.baud_rate = 9600                   # Set the baud rate for SM_VOUT to 9600, ensuring both instruments communicate at the same speed
print(SM_VOUT)                             # Print the resource object (SM_VOUT) to display its properties
print(SM_VOUT.query('*IDN?'))              # Send the '*IDN?' command to the SM_VOUT instrument and print its response, identifying the instrument

#Establishing Connection for VREF multimeter
MM_VREF = rm.open_resource('USB0::10893::257::MY57508183::0::INSTR')
MM_VREF.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
MM_VREF.write_termination = '\n'
MM_VREF.baud_rate = 9600
print(MM_VREF)                             # Print the resource object (SM_VOUT) to display its properties
print(MM_VREF.query('*IDN?'))              

#Establishing Connection for VREF waveform generator
WG_VREF = rm.open_resource('USB0::2391::22279::MY53802060::0::INSTR')
WG_VREF.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
WG_VREF.write_termination = '\n'
WG_VREF.baud_rate = 9600
print(WG_VREF)                             # Print the resource object (SM_VOUT) to display its properties
print(WG_VREF.query('*IDN?'))

#Establishing Connection for Temperature Chamber
CHAMBER = rm.open_resource('GPIB6::6::INSTR') 
CHAMBER.read_termination = '\n'            
CHAMBER.write_termination = '\n'           
CHAMBER.baud_rate = 9600                  
print(CHAMBER)                            
print(CHAMBER.query('STATUS?'))              


VIN_MAX=33
VIN_STEP=1
VIN_MIN=11
ILMAX=-.01
VIN=[]
IQ=[]
VOUT=[]

SerialInstrument at ASRL3::INSTR
KEITHLEY INSTRUMENTS INC.,MODEL 2401,4636506,B02 Jan 20 2021 10:19:49/B01  /W/N
SerialInstrument at ASRL4::INSTR
KEITHLEY INSTRUMENTS INC.,MODEL 2401,4636504,B02 Jan 20 2021 10:19:49/B01  /W/N
USBInstrument at USB0::10893::257::MY57508183::0::INSTR
Keysight Technologies,34465A,MY57508183,A.02.17-02.40-02.17-00.52-04-01
USBInstrument at USB0::2391::22279::MY53802060::0::INSTR
Agilent Technologies,33622A,MY53802060,A.01.11-2.25-03-64-02
GPIBInstrument at GPIB6::6::INSTR
YNNNYYNNNNNNNNNNNN0


<b> Initialize Source Meter (VIN) <b>

In [3]:
SM_VIN.write(":CONF:CURR")                  # Configure the SMU to measure current
print(SM_VIN.query(":CONF?"))               # Query and print the current configuration of the SMU
SM_VIN.write("SOUR:VOLT:RANG 3.3")
SM_VIN.write("SENS:CURR:PROT:LEV 0.0105")   # Set the current protection level (compliance) to 10.5 mA. This limits the maximum current to protect the device under test.
SM_VIN.write("SENS:CURR:RANG 1E-2")         # Set the current measurement range to 10 mA. This sets the expected maximum current for accurate measurement.
SM_VIN.write("CURR:NPLC 10")                # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                            # This controls the integration time, with higher values leading to more accurate but slower measurements.
SM_VIN.write("OUTP:STAT 0")                 # Turn off the VIN source meter

"CURR:DC"


12

<b> Initialize Source Meter (Source/Sink) <b>

In [4]:
SM_VOUT.write(":SOUR:FUNC CURR")             #Configure 
SM_VOUT.write("SOUR:CURR:RANG 0.0105") 
SM_VOUT.write(":CONF:VOLT")                  # Configure the SMU to measure voltage
SM_VOUT.write(":SYST:RSEN 1")                   #Configure SMU to use 4-wire voltage measurement, this minimizes voltage drop along cable
print(SM_VOUT.query(":CONF?"))               # Query and print the current configuration of the SMU

SM_VOUT.write("SENS:VOLT:PROT:LEV 2.5")      # Set the voltage protection level (compliance) to 2.5 V. This limits the maximum voltage to protect the device under test.
SM_VOUT.write("SENS:VOLT:RANG 10")          # Set the voltage measurement range to 10 V. This sets the expected maximum voltage for accurate measurement.
SM_VOUT.write("VOLT:NPLC 10")                # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                             # This controls the integration time, with higher values leading to more accurate but slower measurements.
SM_VOUT.write("OUTP:STAT 0")                 # Turn off the VIN source meter


"VOLT:DC"


12

<b> Initialize Waveform Generator <b>

In [5]:
WG_VREF.write("OUTP:LOAD INF")
WG_VREF.write("SOUR1:APPL:DC")

14

<b> Initialize Multi Meter <b>

In [6]:
MM_VREF.write(":CONF:VOLT:DC")
print(MM_VREF.query(":CONF?"))
print(MM_VREF.query(":READ?"))

"VOLT +1.00000000E+01,+1.00000000E-06"
+1.05542465E-04


<b> Initialize Temperature Chamber <b>

In [7]:
CHAMBER.write("ON")
CHAMBER.write("RATE=30")
CHAMBER.write("WAIT=30")                 #Sets the time that the chamber will wait once the desired temperature is reached before measurements start
CHAMBER.write("HON")
CHAMBER.write("CON")
time.sleep(1) 

print(CHAMBER.query("RATE?"))
print(CHAMBER.query("WAIT?"))

+30.00
00:30:00


<b> Enter the Device under Test <b>

In [8]:
directory_name = input("Enter the Device under Test (DUT):")  # Prompt the user to input the temperature of the experiment, which will be used as the directory name
DUT="DUT_"+str(directory_name)
os.makedirs(DUT, exist_ok=True)                          # Create a directory with the given name. If the directory already exists, it won't raise an error due to exist_ok=True

In [9]:
print(CHAMBER.query("STATUS?"))
print(CHAMBER.query("RATE?"))
#CHAMBER.write("set=100")

YNNNYYNNNNNNNNNNNN0
+30.00


<b> Test Protocol </b>

In [ ]:
VREFlist = [0.8, 1.2, 1.6, 2.0, 2.4]                                                            # List of VREF voltages to test
#TEMPlist = [-140, -160, -175]
TEMPlist = [120, 100, 80, 60, 40, 20, 0, -20, -40, -60, -80, -100, -120, -140, -160, -175]      # List of temperatures for test

# Makes list of load currents to test
ILlist=[]
for j in [-.00001, -.0001, -.001]:
    for j2 in range(1,10):
        ILlist.append(j*j2)

for TEMP in TEMPlist:
    sub_directory_name = TEMP                                   # The temperature of the experiment gets set as the sub directory name
    local=str(DUT)+"/TEMP_"+str(sub_directory_name)
    os.makedirs(local, exist_ok=True)                           # Create a directory with the given name. If the directory already exists, it won't raise an error due to exist_ok=True
    CHAMBER.write("WAIT=30")                                    # Resets the wait time of the temp chamber to 30 minutes so it doesn't time out mid test
    CHAMBER.write("SET=" + str(TEMP))
    while (CHAMBER.query("STATUS?")[3] == 'N'):                 # Checks if the temperature chamber has arrived at TEMP yet or not
        time.sleep(15)
        print(CHAMBER.query("STATUS?"))  
        
    print(CHAMBER.query("STATUS?"))
    time.sleep(120)                                             # settling time for temperature chamber, lets the temperature stabilize
    
    for VREF in VREFlist:                                       # Sets VREF constant throughout test
        WG_VREF.write("APPL:DC DEF, DEF, " + str(VREF))         # Sets VREF, will adjust to be more accurate once source meters are turned on
        
        SM_VIN.write("SENS:CURR:RANG 1E-2")                     # Set the current range to 10 mA before turning anything on, prevents compliance limiting
        SM_VOUT.write("SENS:VOLT:RANG " + str(VREF))            # The voltage at VOUT should be capped by VREF, this will provide the most accurate measurement
        time.sleep(.5) 

        SHEET="VREF="+str(VREF)+".csv"                          # Create a CSV file name based on the VREF value
        VIN=[]                                                  # Initialize an empty list to store VIN values
        IQ=[]                                                   # Initialize an empty list to store IQ (current) values
        VOUT=[]                                                 # Initialize an empty list to store VOUT values
        ILdata=[]
        VREFdata=[]
        IL = ILlist[0]

        ILOAD="SOUR:CURR:LEV:IMM:AMPL " + str(IL)               # Create a string command to set the current level to ILOAD
        print(ILOAD)                                            # Print the ILOAD command for verification
        SM_VOUT.write(ILOAD)                                    # Set the current level on the VOUT source meter

        SM_VIN.write("SOUR:VOLT:LEV:IMM:AMPL 3.3")                  
        SM_VIN.write("OUTP:STAT 1")                             # Turn on the VIN source meter output
        time.sleep(.5)
        SM_VOUT.write(":SOUR:CURR:MODE FIX")                    # Sets current mode to fix in case sweeps are used

        SM_VOUT.write("OUTP:STAT 1")                            # Turn on the VOUT source meter output
        time.sleep(1)
        VREFmeas = MM_VREF.query(":READ?")                      # Set VREF a second time due to changes from turning on source-meters, and adjust it with the multimeter
        WGout = 2*VREF - float(VREFmeas)
        while (abs(float(VREFmeas) - VREF) > .001 *VREF):
            WGout = VREF + WGout - float(VREFmeas)
            WG_VREF.write("APPL:DC DEF, DEF, " + str(WGout))
            time.sleep(1)
            VREFmeas = MM_VREF.query(":READ?")

        ##################### FIRST: Sweep of IL for VIN = 3.3V #####################
        # Current starts low -10uA, sweeps up to -10mA
        #TODO: Change to sweep
        #       Need to set up trigger link to grab VOUT and VIN measurement at the same time
        #       Figure out how to range for that
        for IL in ILlist:
            CURRrange = .00025 + -IL                            # Sets the current range for the VIN sourcemeter. 250uA is roughly the Q current of the OP AMP
                                                                # That plus IL is what should be expected to flow through VIN
            if CURRrange > .01: CURRrange = .01                 # .01 should be the maximum current range used
            SM_VIN.write("SENS:CURR:RANG " + str(CURRrange))   
            time.sleep(.5) 
            ILOAD = "SOUR:CURR:LEV:IMM:AMPL " + str(IL)         # Constructs a string to tell VOUT what to set ILOAD to
            SM_VOUT.write(ILOAD)
            time.sleep(.5)                                      # Wait time before measurement is taken
            VOUT.append(SM_VOUT.query(":READ?").split(',')[0])  # READ is used instead of MEAS as it doesn't reconfigure device, which would turn on auto-range
            IQ.append(SM_VIN.query(":READ?").split(',')[1])     # Query and append the measured IQ current
            ILdata.append(IL)
            VREFdata.append(MM_VREF.query(":READ?"))                                   
            VIN.append(3.3)                                     # Append the current VIN value to the VIN list



        ##################### SECOND: Sweep of VIN for IL=10mA #####################
        # VIN starts high at 3.3V and then sweeps down to 1V or VREF, whichever is higher.
        ILMAX = -.01                                            # Sets ILMAX to -.01 which is the value to be used for the test
        IL = ILMAX                                              # The variable IL is used to set values for consistency
        CURRrange = .01                                         
        SM_VIN.write("SENS:CURR:RANG " + str(CURRrange))        # Sets the measure current range for VIN, it will always be 10mA for this test
        ILOAD = "SOUR:CURR:LEV:IMM:AMPL " + str(IL)
        SM_VOUT.write(ILOAD)                                    # Sets the Vout source meter current to 10mA
        time.sleep(.5)

        if VIN_MIN < VREF*10:                                   # Checks if the VIN_MIN value set in the first code block is less than VREF
            new_MIN = VREF*10 - 1                               # If yes: set the new minimum to be VREF instead of VIN_MIN
        else: new_MIN = VIN_MIN                                 # If no: continue using VIN_MIN
        for i in range (VIN_MAX,int(new_MIN),-VIN_STEP):        # Loop from VIN_MAX to VIN_MIN in steps of VIN_STEP                                                            
            val = i / 10                                        # Convert the current loop index into a floating-point voltage value
            ############ Sets VIN voltage ############
            VIN_V = "SOUR:VOLT:LEV:IMM:AMPL " + str(val)                # Create the command string to set VIN voltage
            SM_VIN.write(VIN_V)                                         # Write the command to set the VIN voltage on the source meter
            SM_VIN.write("SOUR:VOLT:RANG " + str(val))                  # Set the VIN source meter voltage source range to what is being output
                                                                            # Sets the range after new value is set because val is sweeping down
            time.sleep(1)                                               # Pause for 1 second to allow the system to stabilize
            
            ######## Measure ILOAD and others ########
            VOUT.append(SM_VOUT.query(":READ?").split(',')[0])     
            IQ.append(SM_VIN.query(":READ?").split(',')[1])             # Query and append the measured IQ current
            VREFdata.append(MM_VREF.query(":READ?"))
            ILdata.append(IL)
            VIN.append(val)                                          # Append the current VIN value to the VIN list
        SM_VIN.write("SOUR:VOLT:RANG 3.3")
        # After the loop completes, turn off the source meters
        SM_VOUT.write("OUTP:STAT 0")                                 # Turn off the VOUT source meter
        SM_VIN.write("OUTP:STAT 0")                                  # Turn off the VIN source meter

        # Create a DataFrame with the collected data
        df = dp.DataFrame({'VIN': VIN, 'IQ': IQ, 'IL': ILdata, 'VOUT': VOUT, 'VREF': VREFdata})      # Create a pandas DataFrame from VIN, IQ, and VOUT lists

        csv_path=os.path.join(local, SHEET)                          # Generate the file path for saving the CSV
        df.to_csv(csv_path, index=False)                             # Save the DataFrame to a CSV file without including the index
CHAMBER.write("SET=25")                             # Sets the temperature back to room temp once test is complete
WG_VREF.write("OUTP:STAT 0")                        
while (CHAMBER.query("STATUS?")[3] == 'N'):         # Checks if chamber has arrived at room temp
    time.sleep(15)
CHAMBER.write("STOP")                               # Once chamber is at room temp, chamber stops operation

YNNYYYYNNNNNNNNNNN0
YNNYYYYNNNNNNNNNNN0
SOUR:CURR:LEV:IMM:AMPL -1e-05
SOUR:CURR:LEV:IMM:AMPL -1e-05
SOUR:CURR:LEV:IMM:AMPL -1e-05
SOUR:CURR:LEV:IMM:AMPL -1e-05


In [ ]:
SM_VOUT.write("OUTP:STAT 0")                                 # Turn off the VOUT source meter
SM_VIN.write("OUTP:STAT 0") 
WG_VREF.write("OUTP:STAT 0")

In [ ]:
#CHAMBER.write("SET=25")
CHAMBER.write("STOP")